In [2]:
import numpy as np
import matplotlib.pylab as plt

In [6]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

    
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [4]:
apple = 100
apple_num = 2
tax =1.1

#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#forword
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [5]:
#backward
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_price, dtax, dapple, dapple_num)

1.1 200 2.2 110.00000000000001


In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax =1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()


#forword
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)

total_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(total_price, tax)

#backward
dprice = 1
dtotal_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dtotal_price)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple_price, dorange_price, dtax, dapple, dapple_num, dorange, dorange_num)

715.0000000000001
1.1 1.1 650 2.2 110.00000000000001 3.3000000000000003 165.0


# 5.5 活性化レイヤの実装

In [2]:
#入力はnumpy配列を想定
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [3]:
x = np.array([[1., -1.],[2.,-3.5]])
relu = Relu()
relu.forward(x)
relu.mask


array([[False,  True],
       [False,  True]])

In [4]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1. - self.out) 
        return dx

# Affine/Softmaxレイヤの実装

In [3]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

In [4]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [5]:
X_dot_W+B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [6]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis = 0)
        return dx